# Running an simulaulation model of a queueing system without priorities

The simulation model of a queueing system supports various types of input and service distributions, any number of channels. It is possible to specify the maximum length of the queue. By default, it is an infinite queue. In Kendall's notation: GI/G/n/r and GI/G/n

To run the simulation model, it requires 4 steps:
- creating an instance of the simulation model
- specifying the source distribution
- specifying the service distribution
- running the model


#### Import the QueueingSystemSimulator class and create an instance of the system. 
When creating it, specify the number of channels *n* and the maximum queue length *r*. If not specified, the queue is considered unlimited by default.

In [ ]:
from most_queue.sim.queueing_systems.fifo import QueueingSystemSimulator

n = 3
r = 100

qs = QueueingSystemSimulator(n, buffer=r)

#### Setting the input stream.
The *set_sources* method takes two parameters:
- distribution parameters
- type of distribution

For exponential distribution, you only need to pass one parameter - the intensity of the incoming flow. For most distributions, parameters are specified as a list.

#### Supported types of distributions
| Distribution Type | Type | Parameters |
| --- | --- | --- |
| Exponential | 'M' | mu
| Hyper-exponential 2nd order | 'N' | [y1, mu1, mu2]
| Gamma distribution | 'Gamma' | [mu, alpha]
| Erlang | 'E'| [r, mu]
| Cox 2nd order | 'C' | [y1, mu1, mu2]
| Pareto | 'Pa'| [alpha, K]
| Deterministic | 'D' | [b]
| Uniform | 'Uniform'| [mean, half_interval]
| Normal | 'Norm'| [mean, standard_deviation]

In [2]:
l = 1.0
qs.set_sources(l, 'M')

#### Set the service time distribution. Calculate the service intensity through the utilization coefficient and the number of channels

In [3]:
ro = 0.8  # коэффициент загрузки
mu = l / (ro * n)  # интенсивность обслуживания
qs.set_servers(mu, 'M')

#### To run the simulation, you need to call the *run* method and pass the number of jobs

In [4]:
num_of_jobs = 1000000
qs.run(num_of_jobs)

Start simulation


Job served:    | 0/100 [00:00<?, ?it/s]10000/1000000:   1%|          | 1/100 [00:00<00:09, 10.81it/s]10000/1000000:   2%|▏         | 2/100 [00:00<00:08, 11.26it/s]20000/1000000:   2%|▏         | 2/100 [00:00<00:08, 11.26it/s]30000/1000000:   3%|▎         | 3/100 [00:00<00:08, 11.26it/s]30000/1000000:   4%|▍         | 4/100 [00:00<00:08, 11.09it/s]40000/1000000:   4%|▍         | 4/100 [00:00<00:08, 11.09it/s]50000/1000000:   5%|▌         | 5/100 [00:00<00:08, 11.09it/s]50000/1000000:   6%|▌         | 6/100 [00:00<00:08, 11.15it/s]60000/1000000:   6%|▌         | 6/100 [00:00<00:08, 11.15it/s]70000/1000000:   7%|▋         | 7/100 [00:00<00:08, 11.15it/s]70000/1000000:   8%|▊         | 8/100 [00:00<00:08, 11.23it/s]80000/1000000:   8%|▊         | 8/100 [00:00<00:08, 11.23it/s]90000/1000000:   9%|▉         | 9/100 [00:00<00:08, 11.23it/s]90000/1000000:  10%|█         | 10/100 [00:00<00:08, 11.17it/s]100000/1000000:  10%|█         | 10/100 [00:00<00:08, 11.17it/s]110000/1000000:  11%|█      

Simulation is finished



#### After completing the simsimulation, you can obtain the initial moments of waiting and staying times, as well as the probabilities of system states.

In [5]:
print("Time spent ", qs.time_spent)
v_sim = qs.v
w_sim = qs.w
p_sim = qs.p
print(v_sim)

Time spent  9.502973882
[4.955212396950723, 43.78583463893342, 546.6101232857176]


#### Compare the results of simuimulation with the results of numerical calculation for the model M/M/n/r

In [ ]:
from most_queue.theory.queueing_systems.fifo.mmnr_calc import MMnrCalc
from most_queue.general.tables import times_print, probs_print

mmnr = MMnrCalc(l, mu, n, r)
w = mmnr.get_w()
times_print(w_sim, w)

Начальные моменты времени ожидания заявок в системе:
   № момента   |     Числ      |      ИМ       
---------------------------------------------
       1       |    2.5888     |    2.5525     
       2       |     20.71     |    19.964     
       3       |    248.52     |    230.95     



#### Check for correctness of simulation results with the results of calculation for the model M/D/n

In [ ]:
from most_queue.theory.fifo.m_d_n_calc import MDn

qs = QueueingSystemSimulator(n)

qs.set_sources(l, 'M')
qs.set_servers(1.0 / mu, 'D')

qs.run(num_of_jobs)

mdn = MDn(l, 1 / mu, n)
p_ch = mdn.calc_p()
p_im = qs.get_p()

print("Time spent ", qs.time_spent)

probs_print(p_im, p_ch, 10)